---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)



,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)



### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    
    per_doc = len(spam_data[spam_data['target']==1])
    total = len(spam_data['target'])
    return per_doc*100/total

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vectorizer = CountVectorizer()
    vecto = vectorizer.fit(X_train)
    tokens = vecto.get_feature_names()
    tokens = sorted(tokens, key=len)
    
    return tokens[len(tokens)-1]

In [6]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vectorizer = CountVectorizer()
    vecto = vectorizer.fit(X_train)
    vecto_X_train=vecto.transform(X_train)
    
    clfrNB=MultinomialNB(alpha=0.1)
    clfrNB.fit(vecto_X_train, y_train)
    
    y_test_predicted = clfrNB.predict(vecto.transform(X_test))
    auc_score_test = roc_auc_score(y_test, y_test_predicted)

    return auc_score_test

In [8]:
answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vectorizer = TfidfVectorizer()
    vectorizer.fit_transform(X_train)

    feature_names = vectorizer.get_feature_names()

    idf_value = vectorizer.idf_
    
    features_idfs = list(zip(feature_names, idf_value))

    smallest_tf_idf = sorted(features_idfs, key=lambda x: x[1])[:20]

    smallest_tf_idf = pd.Series([features[1] for features in smallest_tf_idf], index=[features[0] for features in smallest_tf_idf])

    largest_tf_idf = sorted(features_idfs, key=lambda x: x[1], reverse=True)[:20]
    largest_tf_idf = pd.Series([features[1] for features in largest_tf_idf], index=[features[0] for features in largest_tf_idf])
    
    return (smallest_tf_idf, largest_tf_idf)


In [10]:
answer_four()

(to      2.198406
 you     2.265645
 the     2.707383
 in      2.890761
 and     2.976764
 is      3.003012
 me      3.111530
 for     3.206840
 it      3.222174
 my      3.231044
 call    3.297812
 your    3.300196
 of      3.319473
 have    3.354130
 that    3.408477
 on      3.463136
 now     3.465949
 can     3.545053
 are     3.560414
 so      3.566625
 dtype: float64,
 000pes         8.644919
 0089           8.644919
 0121           8.644919
 01223585236    8.644919
 0125698789     8.644919
 02072069400    8.644919
 02073162414    8.644919
 02085076972    8.644919
 021            8.644919
 0430           8.644919
 07008009200    8.644919
 07099833605    8.644919
 07123456789    8.644919
 0721072        8.644919
 07753741225    8.644919
 077xxx         8.644919
 078            8.644919
 07808247860    8.644919
 07808726822    8.644919
 078498         8.644919
 dtype: float64)

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
def answer_five():
    vectorizer = TfidfVectorizer(min_df=3).fit(X_train)
    x_train_transformed = vectorizer.transform(X_train)
    x_test_transformed=vectorizer.transform(X_test)
    
    
    clfrNB=MultinomialNB(alpha=0.1)
    clfrNB.fit(x_train_transformed, y_train)
    
    y_test_predicted = clfrNB.predict(x_test_transformed)
    auc_score_test = roc_auc_score(y_test, y_test_predicted)
    
    return auc_score_test

In [12]:
answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [13]:
def answer_six():
 
    non_spam_doc=spam_data[spam_data['target']==0]
    length_non_spam=np.mean([len(x) for x in non_spam_doc.text])
    spam_doc=spam_data[spam_data['target']==1]
    length_spam=np.mean([len(x) for x in spam_doc.text])
    tup = (length_non_spam, length_spam)
    return tup

In [14]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [16]:
from sklearn.svm import SVC

def answer_seven():
    vectorizer = TfidfVectorizer(min_df=5).fit(X_train)
    x_train_transformed = vectorizer.transform(X_train)
    
    x_train_transformed_added= add_feature(x_train_transformed, [len(x) for x in X_train])
    
    x_test_transformed=vectorizer.transform(X_test)
    x_test_transformed_added = add_feature(x_test_transformed, [len(x) for x in X_test])
    
    SVC_model=SVC(C=10000)
    SVC_model.fit(x_train_transformed_added, y_train)
    
    y_test_predicted = SVC_model.predict(x_test_transformed_added)
    auc_score_test = roc_auc_score(y_test, y_test_predicted)
    
    return auc_score_test

In [17]:
answer_seven()

0.9661689557407943

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [48]:
def answer_eight():
    non_spam_doc=spam_data[spam_data['target']==0]
    
    num_not_spam_doc= [sum(char.isdigit() for char in x) for x in non_spam_doc.text]
    num_not_spam_doc= np.mean(num_not_spam_doc)
    
    spam_doc=spam_data[spam_data['target']==1]
    num_spam_doc= [sum(char.isdigit() for char in x) for x in spam_doc.text]
    num_spam_doc =np.mean(num_spam_doc)
    
    return (num_not_spam_doc,num_spam_doc)

In [49]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [81]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    vectorizer = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    
    x_train_transformed = vectorizer.transform(X_train)
    
    X_train_len=[len(x) for x in X_train]
    X_test_len=[len(x) for x in X_test]
    
    X_train_features= [sum(char.isdigit() for char in x) for x in X_train]

    X_test_features= [sum(char.isdigit() for char in x) for x in X_test]
    
    
    X_train_transformed_lengthOfDoc= add_feature(x_train_transformed, [X_train_len,X_train_features])
    
    X_test_transformed=vectorizer.transform(X_test)
    X_test_transformed_lengthOfDoc = add_feature(X_test_transformed,[X_test_len,X_test_features]) 
    
    LR_model=LogisticRegression(C=100, max_iter=10000)
    LR_model.fit(X_train_transformed_lengthOfDoc, y_train)
    
    y_test_predicted = LR_model.predict(X_test_transformed_lengthOfDoc)
    auc_score_test = roc_auc_score(y_test, y_test_predicted)
    
    return auc_score_test

In [82]:
answer_nine()

0.9678709064054463

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [64]:
import re
def answer_ten():
    non_spam_doc=spam_data[spam_data['target']==0]
    num_not_spam_doc= np.mean(non_spam_doc.text.str.count('(\W)'))

    spam_doc=spam_data[spam_data['target']==1]
    
    num_spam_doc =np.mean(spam_doc.text.str.count('(\W)'))
    
    return (num_not_spam_doc,num_spam_doc)

In [66]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [93]:
def answer_eleven():
    vectorizer = TfidfVectorizer(min_df=5,ngram_range=(2,5),analyzer='char_wb').fit(X_train)
    
    x_train_transformed = vectorizer.transform(X_train)
    
    X_train_features= [sum(char.isdigit() for char in x) for x in X_train]
    
    X_test_features= [sum(char.isdigit() for char in x) for x in X_test]
    
    X_train_len=[len(x) for x in X_train]
    X_test_len=[len(x) for x in X_test]
    
    num_non_word_train= X_train.str.count('(\W)')
    num_non_word_test= X_test.str.count('(\W)')
    

    X_train_transformed_lengthOfDoc= add_feature(x_train_transformed, [X_train_len,X_train_features,num_non_word_train])
    
    X_test_transformed=vectorizer.transform(X_test)
    
    X_test_transformed_lengthOfDoc = add_feature(X_test_transformed,[X_test_len,X_test_features,num_non_word_test]) 
    
    LR_model=LogisticRegression(C=100, max_iter=10000)
    LR_model.fit(X_train_transformed_lengthOfDoc, y_train)
    
    y_test_predicted = LR_model.predict(X_test_transformed_lengthOfDoc)
    auc_score_test = roc_auc_score(y_test, y_test_predicted)
    
    feature_names = np.array(vectorizer.get_feature_names()+['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coeff_index=LR_model.coef_[0].argsort()
    smallest_coeffs=feature_names[sorted_coeff_index[:10]].tolist()
    largest_coeffs=feature_names[sorted_coeff_index[:-11:-1]].tolist()
    
    return (auc_score_test, smallest_coeffs,largest_coeffs)

In [1]:
answer_eleven()

NameError: name 'answer_eleven' is not defined